# Logistic regression model with CV and regularisation 

Questions :
- lasso or ridge ? given that the number of parameters for now is very small might not be that usefull (also massive training set)

TODO :
- take a validation set
- do CV on the rest (test, train split)
- choose regularisation hyperparameter one validation set performance 
- possibly iterate on validation set selection (make sure they are rather balanced in term of classification)


ISSUES :
- missing values in the eeg : drop or try to replace ?

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd 

## Preamble

In [3]:
from source.preamble import *

train_eeg_names = os.listdir(Dir.eeg_train)
train_spc_names = os.listdir(Dir.spc_train)
len(train_eeg_names), len(train_spc_names)

(17300, 11138)

## Functions definition

In [4]:
from source.classes import Eeg

In [5]:
from source.scoring import compute_wasserstein
from source.train_algos import train_logistic_regression_CV, test_model, predict_probas_test_set
from source.process import open_train_metadata, print_summary_metadata
from source.pre_train import extract_validation_set


## Process metadata

In [6]:
meta_all = open_train_metadata(base_dir)
print_summary_metadata(meta_all)
VOTE_COLS = [c for c in meta_all.columns if c.endswith("vote")]
meta_all.iloc[:10]

Metadata summary :
Len :  106800


,n_sample,percent
expert_consensus,,
GPD,16702,15
GRDA,18861,17
LPD,14856,13
LRDA,16640,15
Other,18808,17
Seizure,20933,19


,eeg_id,eeg_sub_id,eeg_label_offset_seconds,spectrogram_id,spectrogram_sub_id,spectrogram_label_offset_seconds,label_id,patient_id,expert_consensus,seizure_vote,lpd_vote,gpd_vote,lrda_vote,grda_vote,other_vote,n_votes,eeg_length
0,1628180742,0,0.0,353733,0,0.0,127492639,42516,Seizure,1.0,0.0,0.000000,0.0,0.000000,0.000000,3,6
1,1628180742,1,6.0,353733,1,6.0,3887563113,42516,Seizure,1.0,0.0,0.000000,0.0,0.000000,0.000000,3,2
2,1628180742,2,8.0,353733,2,8.0,1142670488,42516,Seizure,1.0,0.0,0.000000,0.0,0.000000,0.000000,3,10
3,1628180742,3,18.0,353733,3,18.0,2718991173,42516,Seizure,1.0,0.0,0.000000,0.0,0.000000,0.000000,3,6
4,1628180742,4,24.0,353733,4,24.0,3080632009,42516,Seizure,1.0,0.0,0.000000,0.0,0.000000,0.000000,3,2
5,1628180742,5,26.0,353733,5,26.0,2413091605,42516,Seizure,1.0,0.0,0.000000,0.0,0.000000,0.000000,3,4
6,1628180742,6,30.0,353733,6,30.0,364593930,42516,Seizure,1.0,0.0,0.000000,0.0,0.000000,0.000000,3,6
7,1628180742,7,36.0,353733,7,36.0,3811483573,42516,Seizure,1.0,0.0,0.000000,0.0,0.000000,0.000000,3,4
8,1628180742,8,40.0,353733,8,40.0,3388718494,42516,Seizure,1.0,0.0,0.000000,0.0,0.000000,0.000000,3,-40
9,2277392603,0,0.0,924234,0,0.0,1978807404,30539,GPD,0.0,0.0,0.454545,0.0,0.090909,0.454545,11,2


In [7]:
eeg = Eeg(Dir.eeg_train, meta_all.iloc[1])
print("EEG sub id : ", eeg.eeg_sub_id)
display(eeg.open()) # the whole EEG
eeg.open_subs() # only the selected subsample

EEG sub id :  1


,Fp1,F3,C3,P3,F7,T3,T5,O1,Fz,Cz,Pz,Fp2,F4,C4,P4,F8,T4,T6,O2,EKG
0,-80.519997,-70.540001,-80.110001,-108.750000,-120.330002,-88.620003,-101.750000,-104.489998,-99.129997,-90.389999,-97.040001,-77.989998,-88.830002,-112.120003,-108.110001,-95.949997,-98.360001,-121.730003,-106.449997,7.920000
1,-80.449997,-70.330002,-81.760002,-107.669998,-120.769997,-90.820000,-104.260002,-99.730003,-99.070000,-92.290001,-96.019997,-84.500000,-84.989998,-115.610001,-103.860001,-97.470001,-89.290001,-115.500000,-102.059998,29.219999
2,-80.209999,-75.870003,-82.050003,-106.010002,-117.500000,-87.489998,-99.589996,-96.820000,-119.680000,-99.360001,-91.110001,-99.440002,-104.589996,-127.529999,-113.349998,-95.870003,-96.019997,-123.879997,-105.790001,45.740002
3,-84.709999,-75.339996,-87.480003,-108.970001,-121.410004,-94.750000,-105.370003,-100.279999,-113.839996,-102.059998,-95.040001,-99.230003,-101.220001,-125.769997,-111.889999,-97.459999,-97.180000,-128.940002,-109.889999,83.870003
4,-90.570000,-80.790001,-93.000000,-113.870003,-129.960007,-102.860001,-118.599998,-101.099998,-107.660004,-102.339996,-98.510002,-95.300003,-88.930000,-115.639999,-99.800003,-97.500000,-88.730003,-114.849998,-100.250000,97.769997
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17995,-144.660004,-147.809998,-129.820007,-129.460007,-157.509995,-124.000000,-124.570000,-94.820000,-153.070007,-121.110001,-86.459999,-132.520004,-138.339996,-128.970001,-71.300003,-114.480003,-86.709999,-114.959999,-81.500000,-20.070000
17996,-140.880005,-153.000000,-129.529999,-129.020004,-154.059998,-131.220001,-128.380005,-95.000000,-140.820007,-114.639999,-84.379997,-115.339996,-119.230003,-114.709999,-70.989998,-92.129997,-79.639999,-116.139999,-81.879997,10.600000
17997,-133.729996,-141.770004,-121.900002,-122.370003,-158.750000,-123.550003,-127.730003,-93.089996,-125.230003,-106.489998,-83.419998,-112.720001,-103.209999,-107.629997,-61.869999,-97.910004,-77.150002,-106.500000,-75.339996,-2.060000
17998,-141.449997,-151.139999,-127.190002,-128.699997,-163.460007,-124.309998,-129.479996,-94.419998,-140.869995,-113.339996,-83.519997,-129.300003,-118.650002,-117.589996,-71.879997,-99.279999,-83.900002,-116.160004,-81.410004,2.820000


,Fp1,F3,C3,P3,F7,T3,T5,O1,Fz,Cz,Pz,Fp2,F4,C4,P4,F8,T4,T6,O2,EKG
1200,-129.460007,-115.580002,-115.129997,-157.440002,-166.979996,-126.260002,-133.429993,-124.349998,-116.629997,-112.110001,-117.040001,-103.169998,-100.070000,-133.970001,-103.449997,-100.059998,-90.040001,-132.460007,-107.230003,-5.900000
1201,-128.740005,-112.330002,-112.059998,-154.830002,-164.440002,-121.230003,-135.380005,-122.269997,-114.930000,-107.370003,-111.059998,-100.480003,-96.330002,-122.019997,-95.400002,-98.309998,-83.660004,-122.070000,-97.180000,9.670000
1202,-128.600006,-112.550003,-107.489998,-148.360001,-154.089996,-114.110001,-129.470001,-115.989998,-123.620003,-104.839996,-100.190002,-104.250000,-106.040001,-116.940002,-96.059998,-96.620003,-91.120003,-127.430000,-93.900002,-3.030000
1203,-123.449997,-106.440002,-108.870003,-148.309998,-158.080002,-118.129997,-133.350006,-117.790001,-106.150002,-95.480003,-97.440002,-93.980003,-90.500000,-109.900002,-86.510002,-93.559998,-86.019997,-121.650002,-92.110001,33.880001
1204,-129.770004,-119.430000,-111.879997,-148.029999,-145.830002,-118.040001,-140.699997,-113.980003,-97.209999,-85.459999,-91.620003,-83.529999,-71.750000,-96.820000,-71.599998,-85.779999,-68.370003,-105.889999,-85.059998,65.309998
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1595,-90.070000,-98.529999,-97.419998,-136.570007,-87.440002,-101.300003,-115.779999,-111.370003,-134.860001,-109.570000,-100.160004,-138.460007,-134.389999,-143.919998,-124.309998,-170.929993,-125.540001,-141.729996,-114.059998,43.110001
1596,-86.930000,-92.750000,-94.720001,-133.809998,-86.809998,-100.919998,-115.639999,-108.589996,-124.709999,-104.870003,-97.480003,-133.229996,-125.589996,-144.429993,-117.500000,-162.220001,-112.820000,-137.320007,-109.839996,43.790001
1597,-79.970001,-87.339996,-93.320000,-130.649994,-84.040001,-102.349998,-118.220001,-103.809998,-112.970001,-96.510002,-94.070000,-119.800003,-109.680000,-118.169998,-103.800003,-159.839996,-105.739998,-121.879997,-100.889999,-24.500000
1598,-82.449997,-92.910004,-94.309998,-127.550003,-83.699997,-99.430000,-114.739998,-97.669998,-129.570007,-103.820000,-90.059998,-128.289993,-120.379997,-126.330002,-112.760002,-163.860001,-106.739998,-123.610001,-95.739998,-44.270000


## Train

In [8]:
# split validation
rest_meta, validation_meta = extract_validation_set(meta_all, ratio=.05)
print_summary_metadata(rest_meta)
print_summary_metadata(validation_meta)

Metadata summary :
Len :  101460


,n_sample,percent
expert_consensus,,
GPD,15830,15
GRDA,17942,17
LPD,14104,13
LRDA,15815,15
Other,17895,17
Seizure,19874,19


Metadata summary :
Len :  5340


,n_sample,percent
expert_consensus,,
GPD,872,16
GRDA,919,17
LPD,752,14
LRDA,825,15
Other,913,17
Seizure,1059,19


In [9]:
max_it = 1000 if KAGGLE else 100
model = train_logistic_regression_CV(rest_meta, VOTE_COLS, max_it=max_it, grade=Grade.certain)

Number of samples without missing values selected :  85
(85, 40) (85,)


c:\miniconda3\envs\hms\Lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
c:\miniconda3\envs\hms\Lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
c:\miniconda3\envs\hms\Lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
c:\miniconda3\envs\hms\Lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
c:\miniconda3\envs\hms\Lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
c:\miniconda3\envs\hms\Lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached 

In [10]:
if not KAGGLE or True:
    predicted_probas = test_model(model, VOTE_COLS, validation_meta)

Number of samples without missing values selected :  4489


In [11]:
if KAGGLE or True:
    target_probas = validation_meta[VOTE_COLS]
    target_probas

In [12]:
ws = compute_wasserstein(predicted_probas, target_probas)
ws

1.35396877378655

## Test

In [13]:
meta_test = pd.read_csv(os.path.join(Dir.root, "test.csv"))

## Submission

In [15]:
sub = predict_probas_test_set(model, meta_test)
sub.to_csv(os.path.join(Dir.out, "submission.csv"))
sub

,seizure_vote,lpd_vote,gpd_vote,lrda_vote,grda_vote,other_vote
eeg_id,,,,,,
3911565283,0.166578,0.166623,0.166555,0.166729,0.166759,0.166756
